<a href="https://colab.research.google.com/github/gosandayo/Exercises_for_Akari/blob/main/Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.7

モデルの学習（3分程度かかります）

In [190]:
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


X, y = make_moons(n_samples = 10000, noise = 0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = DecisionTreeClassifier(random_state = 42)
params = {
    "max_leaf_nodes": list(range(2,100)),
    "max_depth": list(range(1,7)),
    "min_samples_split": list(range(2,7))
}


In [ ]:

grid_search = GridSearchCV(model, params, cv = 5, scoring = "accuracy")
grid_search.fit(X_train, y_train)
param = grid_search.best_params_
grid_search.best_estimator_

In [191]:
grid_search.best_score_

0.8564999999999999

↑悪くない．

テストしてみる．

In [192]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.854

85-87%に入ったのでよしとする．

# 6.8

In [193]:
from sklearn.model_selection import ShuffleSplit

n_subsets = 1000
n_mini = 100
mini_sets = []
rs = ShuffleSplit(n_splits = n_subsets, test_size = len(X_train) - n_mini, random_state = 42)

In [194]:
for mini_train_index, mini_test_index in rs.split(X_train):
  X = X_train[mini_train_index]
  y = y_train[mini_train_index]
  mini_sets.append((X, y))

In [195]:
accuracy_scores = []

for X_tr, y_tr in mini_sets:
  model = DecisionTreeClassifier(max_depth= param["max_depth"], max_leaf_nodes= param["max_leaf_nodes"], min_samples_split = param["min_samples_split"], random_state = 42)
  model.fit(X_tr, y_tr)
  y_pred = model.predict(X_test)
  score = accuracy_score(y_test,y_pred)
  accuracy_scores.append(score)

np.mean(accuracy_scores)



0.7991095

In [196]:
from sklearn.base import clone

forest = [clone(grid_search.best_estimator_) for _ in range(n_subsets)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7991095

模範解答でやってもこうなりますね．


さて，魔法を起こす．

In [197]:
from scipy import stats

Y_pred = []

for X_tr, y_tr in mini_sets:
  model = DecisionTreeClassifier(max_depth= param["max_depth"], max_leaf_nodes= param["max_leaf_nodes"], min_samples_split = param["min_samples_split"], random_state = 42)
  model.fit(X_tr, y_tr)
  y_pred = model.predict(X_test)
  Y_pred.append(y_pred)

y_pred_ens = stats.mode(Y_pred, axis = 0)
y_pred_ens = y_pred_ens[0]
accuracy_score(y_test, y_pred_ens)

0.857

良い感じ？

模範解答

In [199]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances,
                  random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [200]:
from sklearn.base import clone

forest = [clone(grid_search.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7991095

In [201]:
Y_pred = np.empty([n_subsets, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.857